In [2]:
import transformers
from transformers import BertConfig
config = BertConfig()
has_xla = hasattr(config,"xla_device")
print(has_xla)

False


In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Currently logged in as: sjb15298306261 (sjb15298306261-Nanjing University). Use `wandb login --relogin` to force relogin


acc,▁▆▇▆██▇█
loss,▆█▅▂▃▃▁▂
acc,0.77075
loss,0.25984


In [3]:
import torch

# 检查CUDA是否可用
cuda_available = torch.cuda.is_available()

if cuda_available:
    # 获取GPU设备数量
    num_gpu = torch.cuda.device_count()

    # 获取当前使用的GPU索引
    current_gpu_index = torch.cuda.current_device()

    # 获取当前GPU的名称
    current_gpu_name = torch.cuda.get_device_name(current_gpu_index)

    # 获取GPU显存的总量和已使用量
    total_memory = torch.cuda.get_device_properties(current_gpu_index).total_memory / (1024 ** 3)  # 显存总量(GB)
    used_memory = torch.cuda.memory_allocated(current_gpu_index) / (1024 ** 3)  # 已使用显存(GB)
    free_memory = total_memory - used_memory  # 剩余显存(GB)

    print(f"CUDA可用，共有 {num_gpu} 个GPU设备可用。")
    print(f"当前使用的GPU设备索引：{current_gpu_index}")
    print(f"当前使用的GPU设备名称：{current_gpu_name}")
    print(f"GPU显存总量：{total_memory:.2f} GB")
    print(f"已使用的GPU显存：{used_memory:.2f} GB")
    print(f"剩余GPU显存：{free_memory:.2f} GB")
else:
    print("CUDA不可用。")

# 检查PyTorch版本
print(f"PyTorch版本：{torch.__version__}")

import torch
print(f"CUDA版本：{torch.version.cuda}")




CUDA可用，共有 1 个GPU设备可用。
当前使用的GPU设备索引：0
当前使用的GPU设备名称：NVIDIA GeForce RTX 3060 Laptop GPU
GPU显存总量：6.00 GB
已使用的GPU显存：0.00 GB
剩余GPU显存：6.00 GB
PyTorch版本：2.3.1+cu118
CUDA版本：11.8


In [2]:
# 必要的导入
import numpy as np
from typing import List

PHOSPHO_VOCAB = [
    "0",  # 未磷酸化
    "1",  # 磷酸化
]
SIGNALP_VOCAB = [
    "S",
    "I",
    "M",
    "O",
    "T",
    "L",
]
# 定义Tokenizer类
class Tokenizer:
    def __init__(self, labels: List[str] = SIGNALP_VOCAB):
        # build mapping
        # token_ids与字母表一一对应，即0-0,1-1
        token_ids = list(range(len(labels)))
        # 将上面的token_ids与标签建立映射
        self.vocab = dict(zip(labels, token_ids))

    def tokenize(self, text: str) -> List[str]:
        return [x for x in text]

    def convert_token_to_id(self, token: str) -> int:
        try:
            return self.vocab[token]
        except KeyError:
            raise KeyError(f"Unrecognized token: '{token}'")

    def convert_tokens_to_ids(self, tokens: List[str]) -> List[int]:
        return [self.convert_token_to_id(token) for token in tokens]
    
    def splitline_fasta(self, data):
        lines = data.split('\n')
        identifiers = lines[::3]
        # 第二行是蛋白质序列
        sequences = lines[1::3]
        # 第三行是标签
        labels = lines[2::3]
        return identifiers, sequences, labels
    
    def sequence_to_token_ids(self, sequence) -> List[int]:
        tokens = self.tokenize(labels)
        ids = self.convert_tokens_to_ids(tokens)
        return ids

# 创建Tokenizer实例
tokenizer = Tokenizer()

# 定义输入数据
item = "010110"  # 这是一个由0和1组成的字符串
testdata = """>P35583|EUKARYA|NO_SP|0
MLGAVKMEGHEPSDWSSYYAEPEGYSSVSNMNAGLGMNGMNTYMSMSAAAMGGGSGNMSAGSMNMSSYVG
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII"""
# 处理item，生成token_ids

test_tuple = tokenizer.splitline_fasta(testdata)

identifiers = test_tuple[0][0]  # 这里获取第一个标识符
sequences = test_tuple[1][0]    # 这里获取第一个序列
labels = test_tuple[2][0]       # 这里获取第一个标签

print("Identifiers:", identifiers)
print("Sequences:", sequences)
print("Labels:", labels)

token_ids = tokenizer.tokenize(labels)
print("Token_ids_split",token_ids)
token_ids = tokenizer.convert_tokens_to_ids(token_ids)

label_ids = tokenizer.sequence_to_token_ids(labels)

# 生成与token_ids相同形状的输入掩码，全为1
input_mask = np.ones_like(token_ids)

# 构建返回的元组
return_tuple = (
            np.array(token_ids),
            np.array(label_ids),
            np.array(input_mask),
        )

# 打印结果
print("Token IDs:", return_tuple[0])
print("label_ids:", return_tuple[1])
print("Input Mask:", return_tuple[2])



Identifiers: >P35583|EUKARYA|NO_SP|0
Sequences: MLGAVKMEGHEPSDWSSYYAEPEGYSSVSNMNAGLGMNGMNTYMSMSAAAMGGGSGNMSAGSMNMSSYVG
Labels: IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
Token_ids_split ['I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I']
Token IDs: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
label_ids: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Input Mask: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [3]:
# 导入必要的库
import numpy as np
from typing import Dict, Tuple

# 定义新的词汇表
PHOSPHORYLATION_VOCAB = {
    "NON_PHOSPHORYLATED": 0,
    "PHOSPHORYLATED": 1,
}

# 定义 process_SP 函数
def process_SP(
    label_sequence: str,
    aa_sequence: str,
    vocab: Dict[str, int] = None,
) -> np.ndarray:
    """Generate multi-state tag array from phosphorylation label string and sequence.
    Inputs:
        label_sequence: sequence of ground-truth label in phosphorylation format
        aa_sequence: amino acid sequence, same length as label_sequence
        vocab: dict that maps states to their index
    Returns:
        tag_matrix: zero matrix of shape (seq_len, vocab_values) with ones
                    where the respective label is true
    """
    if vocab is None:
        vocab = PHOSPHORYLATION_VOCAB

    vocab_size = len(set(vocab.values()))  # real vocab size, ignore duplicate mappings
    tag_matrix = np.zeros((len(label_sequence), vocab_size))
    print("最初的零矩阵为：")
    print(tag_matrix)

    label_array = np.array([x for x in label_sequence])

    phosphorylated_idx = np.where(label_array == "1")
    non_phosphorylated_idx = np.where(label_array == "0")

    tag_matrix[non_phosphorylated_idx, vocab["NON_PHOSPHORYLATED"]] = 1
    tag_matrix[phosphorylated_idx, vocab["PHOSPHORYLATED"]] = 1

    assert not any(
        tag_matrix.sum(axis=1) == 0
    ), f"Processing failed. There are positions where no state was set active. \n {aa_sequence}"
    
    return tag_matrix

# 测试代码
label_sequence = "0001000000"
aa_sequence = "MEVGQANPLL"
vocab = PHOSPHORYLATION_VOCAB

tag_matrix = process_SP(label_sequence, aa_sequence, vocab=vocab)
print("标签矩阵：")
print(tag_matrix)


最初的零矩阵为：
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
标签矩阵：
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [5]:
import torch
import numpy as np
from typing import List, Any, Dict

# 定义 pad_sequences 函数
def pad_sequences(sequences, pad_value):
    max_len = max(len(seq) for seq in sequences)
    padded_seqs = np.full((len(sequences), max_len), pad_value)
    for i, seq in enumerate(sequences):
        padded_seqs[i, :len(seq)] = seq
    return padded_seqs

# 修改后的 collate_fn 函数
def collate_fn(batch: List[Any]) -> Dict[str, torch.Tensor]:
    # unpack the list of tuples
    input_ids, label_ids, mask = tuple(zip(*batch))

    data = torch.from_numpy(pad_sequences(input_ids, 0))

    # ignore_index is -1
    targets = pad_sequences(label_ids, -1)
    targets = np.stack(targets)
    targets = torch.from_numpy(targets)
    mask = torch.from_numpy(pad_sequences(mask, 0))

    return_tuple = (data, targets, mask)
    


    return return_tuple

# 测试代码
batch = [
    (np.array([1, 2, 3]), np.array([0, 1, 0]), np.array([1, 1, 1])),
    (np.array([4, 5]), np.array([1, 0]), np.array([1, 1])),
]



result = collate_fn(batch)
print("数据：", result[0])
print("目标：", result[1])
print("掩码：", result[2])
if len(result) > 3:
    print("样本权重：", result[3])


数据： tensor([[1, 2, 3],
        [4, 5, 0]], dtype=torch.int32)
目标： tensor([[ 0,  1,  0],
        [ 1,  0, -1]], dtype=torch.int32)
掩码： tensor([[1, 1, 1],
        [1, 1, 0]], dtype=torch.int32)


In [7]:
import torch
import numpy as np
from torch.utils.data import DataLoader

def validate(model: torch.nn.Module, valid_data: DataLoader, args) -> float:
    """Run over the validation data. Average loss over the full set."""
    model.eval()

    all_targets = []
    all_probs = []
    all_pos_preds = []
    all_token_ids = []

    total_loss = 0
    for i, batch in enumerate(valid_data):
        data, targets, input_mask, sample_weights = batch

        data = data.to(device)
        targets = targets.to(device)
        input_mask = input_mask.to(device)
        sample_weights = sample_weights.to(device) if args.use_sample_weights else None

        with torch.no_grad():
            loss, probs, pos_probs, pos_preds = model(
                data,
                targets=targets,
                sample_weights=sample_weights,
                input_mask=input_mask,
            )

        total_loss += loss.mean().item()
        all_targets.append(targets.detach().cpu().numpy())
        all_probs.append(probs.detach().cpu().numpy())
        all_pos_preds.append(pos_preds.detach().cpu().numpy())
        all_token_ids.append(data.detach().cpu().numpy())

    all_targets = np.concatenate(all_targets)
    all_probs = np.concatenate(all_probs)
    all_pos_preds = np.concatenate(all_pos_preds)
    all_token_ids = np.concatenate(all_token_ids)


    return (total_loss / len(valid_data))

# 示例测试代码
class Args:
    use_sample_weights = False
    use_cs_tag = False

args = Args()

# 创建一个简单的模型和数据加载器进行测试
class SimpleModel(torch.nn.Module):
    def forward(self, data, targets=None, sample_weights=None, input_mask=None):
        loss = torch.tensor(0.5)  # 示例损失
        probs = torch.sigmoid(data)  # 示例概率
        pos_probs = probs
        pos_preds = (probs > 0.5).float()
        return loss, probs, pos_probs, pos_preds

model = SimpleModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 创建一些示例数据
data = torch.randn(70, 10)  # 70 个样本，每个样本有 10 个特征
targets = torch.randint(0, 2, (70, 10))  # 70 个标签，二分类
input_mask = torch.ones_like(data)
sample_weights = torch.ones(70)

# 创建数据加载器
dataset = torch.utils.data.TensorDataset(data, targets, input_mask, sample_weights)
valid_data = DataLoader(dataset, batch_size=10)

# 运行验证函数
avg_loss= validate(model, valid_data, args)
print("平均损失：", avg_loss)



平均损失： 0.5


In [9]:
def subset_dataset(
    identifiers: List[str],
    sequences: List[str],
    labels: List[str],
    partition_id: List[int],
):
    """Extract a subset from the complete .fasta dataset based on partition_id"""

    # break up the identifier into elements
    parsed = [element.lstrip(">").split("|") for element in identifiers]
    acc_ids = [elements[0] for elements in parsed]
    partition_ids = [int(elements[-1]) for elements in parsed]

    partition_ids = np.array(partition_ids)

    part_idx = np.isin(partition_ids, partition_id)

    select_idx = part_idx
    assert select_idx.sum() > 0, "This partition id combination does not yield any sequences!"

    # index in numpy, and then return again as lists.
    identifiers_out, sequences_out, labels_out = [
        list(np.array(x)[select_idx]) for x in [identifiers, sequences, labels]
    ]

    return identifiers_out, sequences_out, labels_out

# 示例测试代码
identifiers = ["Q8TF40|EUKARYA|NO_SP|0", "P12345|ARCHAEA|1", "A1B2C3|NEGATIVE|LIPO|2"]
sequences = ["MAPTLFQKLFSKRTGLGAPGRDARDPDCGFSWPLPEFDPSQIRLIVYQDCERRGRNVLFDSSVKRRNEDI",
             "MKQLEDKVEELLSKNYHLENEVARLKKLV",
             "MKTIIALSYIFCLVFADYKDDDDK"]
labels = ["IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII",
          "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII",
          "IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII"]

partition_id = [0, 1]  # 选择分区 ID 为 0 和 1 的数据

identifiers_out, sequences_out, labels_out = subset_dataset(identifiers, sequences, labels, partition_id)

print("筛选后的标识符：", identifiers_out)
print("筛选后的序列：", sequences_out)
print("筛选后的标签：", labels_out)


筛选后的标识符： ['Q8TF40|EUKARYA|NO_SP|0', 'P12345|ARCHAEA|1']
筛选后的序列： ['MAPTLFQKLFSKRTGLGAPGRDARDPDCGFSWPLPEFDPSQIRLIVYQDCERRGRNVLFDSSVKRRNEDI', 'MKQLEDKVEELLSKNYHLENEVARLKKLV']
筛选后的标签： ['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII']


In [10]:
partition_ids = np.array([1, 4, 2])
partition_id = [1, 2, 3]

part_idx = np.isin(partition_ids, partition_id)
print(part_idx)


[ True False  True]


In [8]:
import torch
import re
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os

# 检查文件是否存在
model_file = r"E:\protT5\spiece.model"
if os.path.exists(model_file):
    print("模型文件存在")
else:
    print("模型文件不存在，请检查路径")

class ProteinT5Tokenizer:
    """Wrapper class for Huggingface T5Tokenizer.
    Implements an encode() method that takes care of:
    - Putting spaces between amino acids.
    - Replacing rare/ambiguous amino acids by 'X'.
    """
    
    def __init__(self, *args, **kwargs):
        self.tokenizer = T5Tokenizer.from_pretrained(*args, **kwargs)

    def encode(self, sequence, padding="longest"):
        # Preprocess sequence to ProtTrans format
        sequence = " ".join(sequence)
        sequence = re.sub(r"[UZOB]", "X", sequence)
        
        # Tokenize sequences with T5Tokenizer
        ids = self.tokenizer.batch_encode_plus(
            [sequence],
            add_special_tokens=True,
            padding=padding
        )
        
        input_ids = torch.tensor(ids['input_ids'])
        attention_mask = torch.tensor(ids['attention_mask'])
        
        return input_ids, attention_mask

    def tokenize(self, sequence):
        return self.tokenizer.tokenize(sequence)

    def convert_tokens_to_ids(self, tokens):
        return self.tokenizer.convert_tokens_to_ids(tokens)


# 定义文件路径
model_path = r"E:\\protT5"

# 加载T5分词器
pretrained_tokenizer = T5Tokenizer.from_pretrained(model_path)

# 使用预训练分词器初始化 ProteinT5Tokenizer
tokenizer = ProteinT5Tokenizer(pretrained_tokenizer)

# 加载T5模型
model = T5ForConditionalGeneration.from_pretrained(r"E:\\protT5\\pytorch_model.bin")
# 示例蛋白质序列
sequence = "MCSRVPDSSSVGAGTAGKFVGGRTK"

# 获取输入张量和注意力掩码
input_ids, attention_mask = tokenizer.encode(sequence)

# 生成嵌入
with torch.no_grad():
    embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)

# 提取嵌入表示
emb_0 = embedding_repr.last_hidden_state[0, :7]  # 根据需求调整
print(f"Shape of per-residue embedding of the first sequence: {emb_0.shape}")

# 平均化以获得每个蛋白质的嵌入表示
emb_0_per_protein = emb_0.mean(dim=0)
print(f"Shape of per-protein embedding of the first sequence: {emb_0_per_protein.shape}")


模型文件存在


OSError: Incorrect path_or_model_id: 'T5Tokenizer(name_or_path='E:\\protT5', vocab_size=28, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>'